In [87]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime

Original attempt was to create custom market factors for within the S&P universe. After extensive searching it was found impossible to collect the data required without paying for a service. Therefore an alternative method of evaluating the factor model was required. 

* Chose to implement Asness and Frazzini's HML calculation, and see how it performs not just in 5 factor (as in their literature) but also for 3 factor and 6 factor

Exploring the difference between Fama French factors vs Asness and Frazzini factors as explained in Asness, C.S, and Frazzini, A. (2013), The Devil in HML’s Details. 

Information on paper can be found: `https://www.aqr.com/Insights/Research/Journal-Article/The-Devil-in-HMLs-Details`

Paper can be found: `https://www.iijournalseprint.com/JPM/AQR/Sum13DevilinHMLsDetails05t/index.html`

Data can be found: `https://www.aqr.com/Insights/Datasets/The-Devil-in-HMLs-Details-Factors-Monthly`

The Journal of Portfolio Management, Vol. 39, No. 4, pp. 49-68

# 3 Factor Model FF vs Asness and Frazzini

In [88]:
factorDF = pd.DataFrame()
for factor in ['MKT','HML Devil','HML FF','SMB', 'RF']:
    if factor != "RF":
        factorDF[factor] = pd.read_excel(
            "The Devil in HMLs Details Factors Monthly.xlsx", sheet_name=factor, index_col=0, header=18)['USA']
    else:
        factorDF[factor] = pd.read_excel(
            "The Devil in HMLs Details Factors Monthly.xlsx", sheet_name=factor, index_col=0, header=18)
factorDF = factorDF * 100

In [89]:
def sampleMeanVariant(sigmahat, muhat, rf, kappa):
    e = np.ones((sigmahat.shape[0], 1))
    H = (muhat - rf*e).T @ np.linalg.inv(sigmahat) @ (muhat - rf*e)
    aStar = (kappa-rf)/H * np.linalg.inv(sigmahat) @ (muhat - rf*e)
    return aStar

### Using fake data here bc kais has yet to get S&P returns

* Important to keep in mind here we are exploring monthly data. That goes for asset returns, factor returns, and rf rate. 

In [90]:
msft = pd.read_csv('msft.csv', parse_dates=True, index_col=0)['Adj Close']
aapl = pd.read_csv('aapl.csv', parse_dates=True, index_col=0)['Adj Close']
googl = pd.read_csv('googl.csv', parse_dates=True, index_col=0)['Adj Close']

returns = pd.DataFrame({'msft': msft, 'aapl': aapl, 'google': googl})
returns = returns.pct_change().dropna()
returns

,msft,aapl,google
Date,,,
2013-03-01,0.037658,0.008699,-0.008749
2013-04-01,0.156937,0.000271,0.038253
2013-05-01,0.054381,0.015697,0.056575
2013-06-01,-0.003375,-0.112457,0.010503
2013-07-01,-0.078170,0.141225,0.008383
...,...,...,...
2022-09-01,-0.107376,-0.119756,-0.116152
2022-10-01,-0.003306,0.109551,-0.011918
2022-11-01,0.099125,-0.034629,0.068564


In [91]:
def factorModel(returns, factorDF, rf, kappa, numFactors):
    """ 
    returns - monthly returns dataframe of each equity (m,d)
    factorDF - df of excess market returns, SMB, HML
    rf - series of rf rate
    kappa - targeted expected return for portfolio
    numFactors - the number of factors in the model
    """

    d = returns.shape[1]
    # Compute the excess returns of each asset
    excess_returns = returns.sub(rf,axis=0).values
#     import ipdb
#     ipdb.set_trace()
    # Define arrays to store the beta coefficients and residual variances
    betas = np.zeros((d, numFactors))
    residual_variances = np.zeros((d))

    # Perform the OLS regression for each asset
    for i in range(d):
        # Extract the excess returns of the i-th asset
        excess_asset_returns = excess_returns[:,i]

        # Define the OLS model with the R-rf ~ MF-rf + SMB + HML
        model = sm.OLS(excess_asset_returns,
                       sm.add_constant(factorDF))

        # Fit the model and extract the regression coefficients and residuals
        results = model.fit()
        beta = results.params[1:]
        residuals = results.resid

        # Store the beta coefficients and residual variance for the i-th asset
        betas[i] = beta
        residual_variances[i] = np.var(residuals, ddof=numFactors + 1)
        
    sigmaFactor = np.cov(factorDF.T)
    Dhat = np.diag(residual_variances)
    sigmahat = betas@sigmaFactor@betas.T + Dhat
    muhat = returns.mean(axis=0).values.reshape((-1, 1))
    # pass most recent rf 
    return sampleMeanVariant(sigmahat, muhat, rf[-1], kappa)


### See how FF HML allocation compares to Asness and Frazzini HML allocation in 3 Factor

In [94]:
testFactor = factorDF[['MKT', 'SMB', 'HML Devil']].loc['02/28/2013':'12/31/2022']
returns.set_index(testFactor.index,inplace=True)

In [95]:
factorModel(returns, factorDF[['MKT', 'SMB', 'HML Devil']].loc['02/28/2013':'12/31/2022'], factorDF['RF'].loc['02/28/2013':'12/31/2022'], 
            kappa=.12,numFactors=3)


array([[0.26593822],
       [0.20211585],
       [0.23471591]])

In [96]:
factorModel(df,factorDF[['MKT','SMB','HML FF']],factorDF['RF'],
            kappa=.12, numFactors=3)


array([[0.19599531],
       [0.22789755],
       [0.26422614]])

### 5 or 6 factor implementation should be identical and should be able to actually use the same function

rolling window


get return 1 time period step ahead

array of returns each time periods

n years of previous data
n+1

n = 10 only roll for 10 years, stopped

do rolling averages of 10-year average... etc

# Things to Look into:

1. Devil vs FF performance in 3 factor
2. Devil vs FF performance in 5 factor (use FF data for all 4 factors and just swap in devil vs not devil)
3. Devil vs FF performance in 6 factor (5 factor with momentum factor)
4. What is a good rolling window to use for calculation? 5 years back? 2 years? 1 year? how does our window choice effect returns? What is the optimal Window?
    * What does this say on how long the market stays constant for?


# Needed from code perspective:
* Overall loop to set lookback and run rolling window
* Data for 5 and 6 factor models
* Returns calculation given allocation provided by `factorModel`
* Performance Metrics 